# Lab 0.2: Calibration, Stacking, and Pretty Pictures!

*Due: Wednesday, April 9 before observing. We will spend the first computer lab working on this notebook.*

This lab walks through calibration and stacking of raw images from the telescope. By the end of this lab, you will be able to
 - perform image calibration using darks, flats, and biases
 - analyze the performance of the camera and telescope using these files
 - align and stack multiple exposures of the same object
 - combine images from individual filters to make a pretty color image

Depending on your background, this lab may also be a crash course in scientific computing with Python, so start this one early! We recommend opening this notebook with Google Colab and making a copy in Google Drive so that you can (1) easily load data from the shared class folder, (2) avoid Python installation and environment headaches, and (3) easily collaborate with your team later in the quarter. If you would like to switch to a different workflow for the final project, we'll happily help you set that up.

You should complete the key conceptual lines of code marked with `TODO:`, answer the **bolded questions**, and read the comments for the rest of the code to make sure you understand what's going on. When you're done, save the notebook as a PDF with cell outputs included and upload to Canvas.

In [ ]:
# If using Colab, mount your Google Drive to access data in the shared folder
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# If using Colab, need to install the reproject package
%pip install reproject

In [ ]:
# Import packages!

# numpy is a core package for numerical computing in Python, mostly it does fast array operations
import numpy as np

# matplotlib is a common plotting library (plotly is another good one)
import matplotlib.pyplot as plt
%matplotlib inline

# astropy is useful for reading FITS files, doing coordinate transformations, converting units, and much more
import astropy
from astropy.io import fits
from astropy.wcs import WCS # World Coordinate System

# reproject allows us to reproject images onto the same coordinates before stacking
from reproject import reproject_interp

# glob and os are useful for navigating your file system
import glob
import os

# miscellaneous
from tqdm.notebook import tqdm
import warnings
from collections import defaultdict

## Calibrating Astronomical Images

Before we jump into the code, here is a quick primer on calibrations for any charge coupled device (or CCD) camera and why they are necessary. The text has been lifted heavily from [Howell (2006, chapter 4)](http://adsabs.harvard.edu/abs/2006hca..book.....H).

### Bias
Every camera offsets the zero level so that the A/D converter never outputs a negative number. This offset is different for every camera and every camera setting (for those cameras with changeable settings). This "bias" value is not necessarily constant for every pixel and must be measured so that it can be subtracted from the raw science frame.

The bias image has an exposure time of zero seconds. The shutter remains closed and the CCD is simply read out. The purpose of a bias or zero frame is to allow the user to determine the underlying noise level within each data frame. The bias value in a CCD image is usually a low spatial frequency variation throughout the array, caused by the CCD on-chip amplifiers. This variation should remain constant with time. The rms value of the bias level is the CCD read noise. A bias frame contains both the DC offset level (overscan) and the variations on that level. The nature of the bias variations for a given CCD are usually column-wise variations, but may also have small row-wise components as well. Thus, a 2-D, pixel-by-pixel subtraction is often required. A single bias frame will not sample these variations well in a statistical fashion, so an average bias image of 10 or more single bias frames is recommended.

### Dark
Photons are emitted from every body with a non-zero temperature (on the Kelvin scale). Therefore everything emits light, even your camera. These thermal photons can cause a signal in a sensitive CCD camera and therefore the rate at which this occurs needs to be accounted for in a fully calibrated image.

CCD dark frames are images taken with the shutter closed but for some time period, usually equal to that of your target frames. That is, if one is planning to dark correct a 45 second exposure, a 45 second dark frame would typically be obtained. Longer dark frames can often be avoided using the assumption that the dark current increases linearly with time and a simple scaling can be applied. However, this is not always true. Dark frames are a method by which the thermal noise (dark current) in a CCD can be measured. They also can give you information about bad or "hot" pixels that exist as well as provide an estimate of the rate of cosmic ray strikes at your observing site. Observatory class CCD cameras are usually cooled with liquid nitrogen to temperatures at which the dark current is essentially zero. Many of these systems therefore do not require the use of dark exposure CCD frames in the calibration process. Thermoelectrically cooled systems are typically not cooled to low enough temperatures such that one may ignore the dark current, but they are getting better. In addition, these less expensive models often have poor temperature stability allowing the dark current to wander a bit with time. Multiple darks averaged together are the best way to produce the final dark calibration frame. Note that the bias is also present in dark frames. To get an accurate measure of the dark current, the bias must be subtracted.

### Flat
Not every pixel in a CCD camera responds to the sky in the exact same way. There are many reasons for this. For one, not every pixel is created equally, so there may be some intrinsic sensitivity difference from pixel to pixel. However, the CCD may be obscured slightly by the optics of the telescope, or by dust on the camera or filters. Therefore this pixel-to-pixel variation must also be accounted for.

Flat field exposures are used to correct for pixel-to-pixel variations in the CCD response as well as any nonuniform illumination of the detector itself. Flat fields expose the CCD to light from either a dome screen, the twilight sky, the nighttime sky, or a projector lamp in an attempt to provide a high signal-to-noise ratio (SNR) uniformly illuminated calibration image. For narrow-band imaging, flats are very helpful in removing fringing, which may occur in object frames. Flat field calibration frames are needed for each color, wavelength region, or different instrumental setup used in which object frames are to be taken. A good flat should remain constant to about 1%, with 2% or larger changes being indicators of a possible problem. As with the other calibration frames, at least 5 or more flat fields should be taken and averaged to produce the final flat used for image calibration.

In [ ]:
# Specify data directory, and which object we want from which date
data_dir = '/content/drive/MyDrive/PHYSICS100_S2025/data' # add a shortcut to PHYSICS100_S2025 in your Google Drive for easy access
object_name = 'm51' # TODO: change this to the object you observed, although we recommend completing the notebook with the M51 example first
date_string = '2024_04_28'
telescope_name = '0.7m'

# Gather all the filenames for the object and dates
filenames = {k: glob.glob(os.path.join(data_dir, 'calibration', telescope_name, date_string, f'{k}*.fit'))for k in ['flat', 'dark', 'bias']} # calibration exposures
filenames['light'] = glob.glob(os.path.join(data_dir, object_name, telescope_name, date_string, '*.fit')) # raw science exposures

In [ ]:
# Sort the data and headers into dictionaries (defaultdicts are like dictionaries, but they create new keys automatically)
data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
headers = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

# For each type of exposure, load all the files and append them to the appropriate lists
for k in filenames.keys():
    for fn in filenames[k]:
        with fits.open(fn) as hdul:
            filter = hdul[0].header['FILTER'] if 'FILTER' in hdul[0].header else None # get filter from header
            exp_time = hdul[0].header['EXPTIME'] # get exposure time from header
            data[k][filter][exp_time].append(hdul[0].data) # append data to the appropriate list
            headers[k][filter][exp_time].append(hdul[0].header) # append header to the appropriate list

In [ ]:
# This is how we access the first flat data for filter R and exposure time 0.2 seconds
data['flat']['R'][0.2][0]

# This is how we access the second light header for filter V and exposure time 30 seconds
headers['light']['V'][30.0][1]

# This is how we see which exposure times are in the dictionary for darks (`None` is because the dark has no filter)
data['dark'][None].keys()

**Question: Why do we need 0.1 s, 0.2 s, and 30 s darks in the example M51 observations? Why do biases not have a filter or exposure time?**

Answer:

In [ ]:
# Compute average bias
bias_frames = np.array(data['bias'][None][0.0]) # Bias has no filter and 0.0s exposure time
avg_bias = # TODO: Compute the average bias frame (hint: use np.mean with the axis argument)

**Question: What is the shape of the `bias_frames` array? What does each dimension represent?** (Hint: use `.shape`)

Answer:

In [ ]:
# Look at the average bias
plt.imshow(avg_bias, cmap='gray', vmin=, vmax=) # TODO: Set vmin and vmax so that you can see the structure in the image
plt.show()

**Question: What is the typical value of the bias? Why does it look noisy? Is there any structure to the noise?**

Answer:

In [ ]:
# Compute average dark for each exposure time (note: if some exposure times were missing, you would have to scale an existing dark appropriately)
avg_darks = {}
for time, darks in data['dark'][None].items(): # Dark has no filter
    avg_darks[time] = # TODO: compute the average dark and subtract the average bias

In [ ]:
# Make a histogram of the dark current at different exposure times
hist_range = # TODO: Set the range of the histogram to ignore the outliers
plt.hist(avg_darks[0.1].flatten(), range=hist_range, bins=20, histtype='step', label='0.1 s')
plt.hist(avg_darks[0.2].flatten(), range=hist_range, bins=20, histtype='step', label='0.2 s')
plt.hist(avg_darks[30].flatten(), range=hist_range, bins=20, histtype='step', label='30 s')
plt.legend()
plt.show()

**Question: What is the typical value of the dark current and do you see any difference between exposure times? What does this tell you? What do you think the outliers are?**

Answer:

In [ ]:
# Compute median flat for each filter
median_flats = {}

for filter in data['flat'].keys():
    all_normed_flats = []
    for time, flats in data['flat'][filter].items():
        corrected_flats = # TODO: subtract average bias and corresponding dark from each flat
        normed_flats = # TODO: normalize flats so that they have a median value of 1.0 (hint: use axis=(1,2) and keepdims=True)
        all_normed_flats.append(normed_flats)
    all_normed_flats = np.concatenate(all_normed_flats, axis=0)
    median_flats[filter] = # TODO: compute the median flat for each filter

**Question: Why do we use the median for the flats but average for darks and biases?**

Answer:

In [ ]:
# TODO: Plot the median flat for each filter, like we did for the bias
fig, axes = plt.subplots(ncols=3, figsize=(15,5))
for i, (filter, flat) in enumerate(median_flats.items()):
    axes[i].imshow(flat, cmap='gray')
    axes[i].set_title(filter)
plt.show()

**Question: What do you see in the flats? Explain what causes this and why it has that shape.**

Answer:

In [ ]:
# Calibrate all science images
calibrated_data = defaultdict(lambda: defaultdict())

for filter, dict in data['light'].items():
    for time, lights in dict.items():
        dark = # TODO: get the right dark for this exposure time
        flat = # TODO: get the right flat for this filter
        calibrated_lights = # TODO: correct for bias, dark, and flat (hint: two are subtracted, one is divided)
        calibrated_data[filter][time] = calibrated_lights

**Question: In the cell above, which are subtracted, which are divided, and why?**

Answer:

## Stacking Images

Now we will step you through the basic procedure to stack images (or "coadd") to achieve a higher signal-to-noise ratio (SNR). We will use the calibrated images we just created above. Individually, it may be difficult to pick out the faint details from a single image, but by stacking the images we should obtain a "deeper" image that shows more subtle detail including fainter objects in the field.

Unfortunately, image stacking can often be a more difficult process than it initially seems. Objects on the night sky are moving, and while the telescopes make a concerted effort to track these movements, the positions of your objects may not be constant across many frames.

What we are going to do is align every image with a reference image. What should we choose as our reference image? There is really no right answer here, but you can imagine that choosing the middle image might make a lot of sense. For instance, if the tracking drifted over the course of obtaining these images, the middle image might be the easiest to align with since it might minimize the amount of shifting for the set of images.

Note that images from the 24" telescope need to be "solved" to add the necessary RA and dec information to their FITS headers. This may be a good idea for images from the 0.7 m as well. If we don't have time to do this at the telescope, you may need to do this yourself with ASTAP (see the addendum).

In [ ]:
# Coadd the calibrated science images
coadded_science = {}
reference_header = headers['light']['V'][30.0][10] # choose one header to serve as the reference to project other images onto
warnings.filterwarnings('ignore', category=astropy.wcs.FITSFixedWarning) # ignore when astropy fixes a FITS file

for filter in calibrated_data.keys():

    # Create arrays to hold the stacked data and exposure time per pixel
    stacked_image = np.zeros(calibrated_data[filter][30.0][0].shape)
    stacked_footprint = np.zeros(calibrated_data[filter][30.0][0].shape)
    total_exposure = 0

    for time, images in calibrated_data[filter].items():
        for i, image in enumerate(tqdm(images, desc=f'Filter {filter} {time} s')): # use tqdm progress bar with custom description

            # TODO: reproject the science image into the same coordinates as the reference image (hint: use reproject_interp, look up the documentation)
            =

            # TODO: update the stacked arrays
            stacked_image +=
            stacked_footprint +=
            total_exposure +=

    # TODO: throw away parts of image not covered by all exposures, by setting to np.nan ("not a number", often used for invalid values)
    # Hint: the code will look something like a[b!=c] = np.nan, this means set all elements of a where b is not equal to c to np.nan
    = np.nan

    # Update header with new exposure time
    new_header = reference_header.copy()
    new_header['EXPTIME'] = total_exposure
    new_header['EXPOSURE'] = total_exposure
    new_header['FILTER'] = filter

    # Final stacked HDU
    coadded_science[filter] = fits.PrimaryHDU(stacked_image, new_header)

**Question: Why is it taking so long? What's so complicated about shifting an image?**

Answer:

In [ ]:
# Plot coadded science images, use log scale to see fainter features
fig, axes = plt.subplots(ncols=3, figsize=(15,5))
for i, (filter, hdu) in enumerate(coadded_science.items()):
    axes[i].imshow(hdu.data, cmap='gray', vmin=np.nanpercentile(hdu.data, 0.1), vmax=np.nanpercentile(hdu.data, 99.9), norm='log')
    axes[i].set_title(filter)
plt.show()

In [ ]:
# If you want, the code below will save the coadded images to wherever you specify
# for filter in coadded_science.keys():
#     hdul = fits.HDUList([coadded_science[filter]])
#     hdul.writeto(os.path.join('<INSERT FOLDER PATH HERE>', 'coadd_{}_{}_{:.0f}s_{}.fit'.format(object_name, filter, coadded_science[filter].header['EXPTIME'], date_string)), overwrite=True)

## Making Color Images

As you may have noticed, the images that we have been working with have all been grayscale. So how do you make a color image? It is a lot more intensive that snapping a shot with your phone, but it also will likely give you much more insight into everything that your phone does.

Matplotlib has the ability to show color images if you specify the levels of Red, Green, and Blue in each pixel (i.e. an array specifying specific colors in an RGB image). For Python to recognize your input as an image, these values must be either decimals in [0,1] or integers in [0,255]. Because the exposures you took of your astronomical object differ in exposure in different bands, it is typical to rescale each color image to these bounds, along with "clipping" the data to account for outliers. To make fainter details more apparent, we can also take the square root or other power of the clipped data. Feel free to try your own limits and scaling fuctions to make the best image possible!

In [ ]:
# Create composite image
composite = np.stack([coadded_science[f].data for f in ['R','V','B']], axis=-1)
clip_min = np.nanpercentile(composite, 80, axis=(0,1)) # minimum at 80th percentile
clip_max = # TODO: Set the maximum at a percentile that looks good
composite = np.clip(composite, clip_min, clip_max)
composite = (composite - clip_min)/(clip_max - clip_min)
composite = # TODO: Take the square root or another fractional power to make the image look better

# Plot
plt.imshow(composite)
plt.axis('off')
plt.gcf().tight_layout()
plt.show()

**Question: What do you notice in your final composite image? No right or wrong answer here and questions are as good as answers!**

Answer:

## Addendum: Plate Solve with ASTAP (no work required here)

One funny quirk of our 24" telescope: the computer for the mount (which tracks and calculates astrometry)
and the camera (which you instruct to take your exposures/calibration files) do not communicate with each
other. This means that the FITS file header for your astronomical images do not contain RA and dec
information that is extremely useful when aligning and stacking images together. Even for the 0.7 m telescope, the RA and dec information may not be accurate enough for stacking. While it is possible to
stack images by calculating statistics of reference stars and aligning pixels, we opt for the method of adding
astrometric information to the FITS headers before alignment. We can do this via the Astronomy Stacking
Program (ASTAP). Using this [link](https://www.hnsky.org/astap.htm), you can download the software (available for Windows, Mac, and Linux) as well as a star catalogue (D80 recommended if you have the space; ∼ 1GB) to compare to a reference image.

For each batch of images input to ASTAP, you can select a reference image to use for alignment. There are
multiple methods for solving, though we recommend you select Star or Astrometric alignment. This will
use the downloadable stellar catalogue to match with reference stars in your image, calculating an RA and
dec for each pixel in your image. The FITS header will be overwritten, and images can be easily combined
with traditional FITS handlers in Python. ASTAP also has built-in stacking
and calibration, but we will be doing this in Python to maintain flexibility.
You can find a detailed index of ASTAP functions via their [helpful guide](https://www.hnsky.org/astap.htm#alignment_menu).

We may have time to do the plate solving at the telescope so that each group does not need to do this individually on their own computers. If not, TAs will be available to answer questions on how to use the software during the Computer Labs. Either way, you'll need to do this yourself for the final project so it's a good idea to start learning.